# 爬虫简介

爬虫一般指从网站上获得需要的数据，这个过程其实是建造一个网站的逆向过程。

# 使用Flask建立简单网站

所谓知己知彼，则百战不殆。既然爬虫主要是从网站上爬取数据，那么只有对建造网站的步骤一清二楚，才能在变幻多端的网站环境下，获取想要的数据。

然而实际建立网站的过程是非常繁琐而复杂的，写HTML、CSS仅仅是一个网站的冰山一角，这其中至少涉及到网站架构设计、数据库设计、后台服务设计、前端设计等等，此外根据不同的架构设计，还有很多中间件、缓存数据库等等复杂的设计。

不过，对于比较简单的网站，Python本身就有很多成熟的框架可以方便我们快速搭建一个简单的网站。这其中，Flask由于其比较精美的设计架构以及简单的模板等应用，是非常受欢迎的轻量级Web框架。在这里，我们不妨使用Flask搭建一个最简单的网站（因为在Jupyter里面，所以我把run()给注释掉了，如果需要执行，请直接执行html/web.py）：

In [1]:
from flask import Flask
app = Flask(__name__)
import random

@app.route('/')
def index():
    with open("example4.html") as f:
        html=f.read()
    return html

@app.route('/dynamic')
def dynamic():
    with open("dynamic.html") as f:
        html=f.read()
    return html

@app.route('/dynamic_response')
def dynamic_response():
    return str(random.random())

if __name__ == '__main__':
    #app.run()
    pass

在执行以上命令之前，需要首先使用pip install flask安装Flask。

在上面的程序中，@代表修饰器（decorator），是Python编程的一个高级特性，我们暂且不管。我们需要知道的仅仅是，通过@app.route函数，声明了一个路径，该路径即访问接下来定义的页面的路径。

在@app.route下方，我们定义了几个函数，这几个函数的作用是返回一个字符串，这些字符串会通过网络传递给访问的浏览器。

如果运行html/web.py，会提示：
>  Running on http://127.0.0.1:5000/

此时，如果在浏览器中输入以上网址，服务器就会执行index()函数，该函数会读取example4.html，并将其返回，从而我们在浏览器上就看到了example4.html。

同理，如果访问 http://127.0.0.1:5000/dynamic ，服务器就会执行dynamic()函数，将dynamic.html的内容返回。

而如果访问 http://127.0.0.1:5000/dynamic_response ，服务器会执行dynamic_response()函数，该函数生成一个随机数并返回给浏览器。

以上就是一个简单的网站。

# 发送请求

知道了服务器端如何处理页面，那么现在的问题是，客户端怎样与服务器端通讯呢？

这里就要介绍以下HTTP协议的概念了。

HTTP协议是**Hyper Text Transfer Protocol**（**超文本传输协议**）的缩写，一种服务器/客户端范式的传输协议，我们访问网址（URL）时，都是以http:// 开头的，或者https:// 开头，代表的就是使用http协议。

一个最简单的传输模型是，客户端通过URL地址向服务器端发送**请求**（**request**），服务器根据所请求的地址、头部信息（headers）做出**响应**（**response**）。

这里有几个概念要特别注意：

## 网址URL

一个常见的网址通常具有如下形式：

> http://econpaper.cn:8080/article/article.jsp?id=56987&userid=3455

其中：

* http:// 代表协议
* econpaper.cn 代表域名
* :8080 代表端口号，默认为80端口
* /article/article.jsp 部分为请求的页面路径
* ?id=56987&userid=3455 为需要传递给这个页面的参数

## 请求（request）

客户端向服务器端发送请求，要按照一定的格式，请求消息由以下三部分组成：

* 请求行（request line）
* 头部（header）
* 请求数据

![](pic/request.png)

比如，以下是一个典型的请求头部：

![](pic/request_headers.png)

这些信息传递给服务器后，服务器根据这些信息进行处理。

实际上，在我们刚刚运行的Flask中，在客户端可以看到每一次的请求。

此外，上面从URL地址中已经看到，?id=56987&userid=3455 为需要传递给这个页面的参数，实际上，这是一个GET的请求方法。为了从客户端向服务器端传数据，以下两种方法是最常用的：

* GET：像上面一样，请求的数据明文写在URL上
* POST：数据包含在请求体中

当然，两种方法也可以结合起来使用。不过最为常用的仍然是GET方法。

## 响应（response）

有了请求，就会有服务器的响应。同样，响应也有响应头和数据体。在响应头中，最重要的是响应的状态码以及内容类型（html文档或者图片、pdf等），常见的状态码比如：

* 200 OK                        客户端请求成功
* 400 Bad Request               客户端请求有语法错误，不能被服务器所理解
* 401 Unauthorized              请求未经授权，这个状态代码必须和WWW-Authenticate报头域一起使用 
* 403 Forbidden                 服务器收到请求，但是拒绝提供服务
* 404 Not Found                 请求资源不存在，eg：输入了错误的URL
* 500 Internal Server Error     服务器发生不可预期的错误
* 503 Server Unavailable        服务器当前不能处理客户端的请求，一段

# Python中发送请求

在Python中，有很多工具可以帮助我们向服务器发送请求，包括但不限于：urllib、urllib2、urllib3、httplib2、requests等等等等。出于个人偏好原因，在这里我们以requests为例，介绍如何发送请求。

为了使用requests，需要先进行安装：pip install requests

使用时直接导入：
```python
import requests
```

接着，最简单的请求即GET请求：
```python
r=requests.get(url)
```

其中url为需要请求的地址。

有时我们在发送请求时可能需要控制发送请求的头部，头部可以使用一个字典表示，比如：
```python
headers={"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36 OPR/60.0.3255.70",
        "Accept-Encoding":"gzip, deflate, lzma, sdch",
        "Accept-Language":"en-US,en;q=0.8",
        "Connection":"keep-alive",
        "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"}
```

以上字典定义了头部的内容，接下来只要在get()函数中加入该头部就可以了：
```python
r=requests.get(url, headers=hearders)
```

返回结果：

* r.text 可以读取返回的HTML
* r.json() 获得Json数据转换后的字典数据
* r.encoding 记录了返回数据的字符编码
* r.status_code 记录了响应的状态码

如果需要发送POST请求，需要首先将请求的数据写成字典形式，在使用urllib包中的urlencode函数将其编码，比如：
```python
data=dict(name="Joe", comment="A test comment")
r=requests.post(url, data=data)
```

如果需要设置自定义的cookie到服务器，可以使用：
```python
cookies = dict(cookies_are='working')
r=requests.get(url, cookies=cookies)
```

如果我们希望设置最长的等待时间，可以使用超时选项：
```python
r=requests.get(url, timeout=10)
```

即设定等待时间超过10s则放弃连接。

最后，如果需要使用代理，可以使用：
```python
proxies = {
  'http': 'http://user@password@10.10.1.10:3128',
  'https': 'http://10.10.1.10:1080',
}
r=requests.get(url, proxies=proxies)
```

比如，使用pm25.in提供的接口爬取pm2.5数据：

In [10]:
url="http://www.pm25.in/api/querys/pm2_5.json?city=shanghai&token=5j1znBVAsnSf5xQyNQyq"
import requests
r=requests.get(url)
data=r.json()
data

{'error': 'Sorry，您这个小时内的API请求次数用完了，休息一下吧！'}

大多数接口都需要用一个token进行身份验证，一般会根据token进行一定的数量限制。

以下代码实现了从链家上爬去房价数据。值得注意的是，如果没有设定headers，可能会爬不下来：

In [8]:
import requests
from bs4 import BeautifulSoup
import re
headers={"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36 OPR/60.0.3255.70",
        "Accept-Encoding":"gzip, deflate, lzma, sdch",
        "Accept-Language":"en-US,en;q=0.8",
        "Connection":"keep-alive",
        "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"}
url='https://sh.lianjia.com/ershoufang/songjiang/'
r=requests.get(url, headers=headers)
html=r.text
bs=BeautifulSoup(html,"html.parser")
iterms=bs.find_all(name='div',attrs={"class":"item"})
info=[]
for it in iterms:
    house_id=it.get("data-houseid")
    prop=it.find(name='div',attrs={"class":"info"}).text
    shi_ting=re.search(r"\d室\d厅",prop).group()
    area=re.search(r"\d*(\.\d*)?平米",prop).group()
    location=prop[:prop.find('/')]
    price=it.find(name='div',attrs={"class":"price"}).text
    print([house_id,shi_ting,area,location,price,prop])

['107102052326', '2室2厅', '83.13平米', '新桥', '245万', '新桥/2室2厅/83.13平米/南/精装']
['107102235592', '3室2厅', '88.74平米', '泗泾', '302万', '泗泾/3室2厅/88.74平米/南 北/精装']
['107102130084', '2室2厅', '89平米', '九亭', '362万', '九亭/2室2厅/89平米/南/精装']
['107102210008', '2室2厅', '90.65平米', '九亭', '378.8万', '九亭/2室2厅/90.65平米/南 北/精装']
['107101778575', '2室1厅', '70.8平米', '泗泾', '202万', '泗泾/2室1厅/70.8平米/南/简装']
['107102269727', '4室2厅', '193.05平米', '松江大学城', '580万', '松江大学城/4室2厅/193.05平米/南 北/简装']
['107101202999', '4室2厅', '186平米', '松江新城', '566万', '松江新城/4室2厅/186平米/南/简装']
['107102035432', '2室1厅', '62.64平米', '松江老城', '177万', '松江老城/2室1厅/62.64平米/南/简装']
['107101355197', '4室2厅', '175.02平米', '莘闵别墅', '685万', '莘闵别墅/4室2厅/175.02平米/南/精装']
['107102250821', '3室2厅', '92.08平米', '松江大学城', '365万', '松江大学城/3室2厅/92.08平米/南/精装']
['107102283498', '3室2厅', '100.28平米', '九亭', '445万', '九亭/3室2厅/100.28平米/南/简装']
['107102299041', '1室1厅', '52.79平米', '泗泾', '155万', '泗泾/1室1厅/52.79平米/南/毛坯']
['107101170742', '2室2厅', '90.91平米', '九亭', '353万', '九亭/2室2厅/90.91平米/南/精装']
['1071022127

接下来我们使用一个更加综合的例子：百度新闻的爬取：

```python
### funcs.py
#!/usr/bin/python3
import requests
import time
from bs4 import BeautifulSoup
## write log file.
def writeLog(content):
    try:
        logfile=open('News.log','a')
        ltime=time.localtime()
        trstr=lambda s:('0'+str(s))[-2:]
        logfile.write(str(ltime.tm_year)+'-'+trstr(ltime.tm_mon)+'-'+trstr(ltime.tm_mday)+' '
                      +trstr(ltime.tm_hour)+':'+trstr(ltime.tm_min)+':'+trstr(ltime.tm_sec)+'-->'+
                      content+'\n')
        logfile.close()
    except Exception as e:
        print(e)
        pass
## get htmltext
def getPage(url,trytimes=10,tot=10):
    trytime=1
    headers={"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36 OPR/60.0.3255.70",
        "Accept-Encoding":"gzip, deflate, lzma, sdch",
        "Accept-Language":"en-US,en;q=0.8",
        "Connection":"keep-alive",
        "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
        }
    while True:
        try:
            r=requests.get(url, headers=headers, timeout=tot, stream=False)
            htmlText=r.text
            break
        except Exception as e:
            if trytime>=trytimes:
                print(e)
                writeLog('Error happened in reading '+url+':'+str(e)+'.')
                return ''
            else:
                trytime=trytime+1
                time.sleep(10)
    return htmlText
# get current date
def getDate():
    ts=time.localtime()
    trstr=lambda s:('0'+str(s))[-2:]
    return '-'.join((str(ts.tm_year),trstr(ts.tm_mon),trstr(ts.tm_mday)))+' '+':'.join((trstr(ts.tm_hour),trstr(ts.tm_min)))
# get the longest content
def getLongText(url):
    html=getPage(url,trytimes=2)
    bs=BeautifulSoup(html,"html.parser")
    content=""
    ## delete all scripts
    while bs.script!=None:
        bs.script.extract()
    while bs.style!=None:
        bs.style.extract()
    ## get content, method 1
    divs=bs.find_all('div')
    text=""
    minlen=5
    for d in divs:
        # if d.find_all("div")==[]:
        text=d.get_text().strip('\n\r ')
        if text is not None and text not in content:
            if text.count('。')>=1 and len(text)>minlen:
                content=content+text
    ## get content, method 2
    ps=bs.find_all('p')
    content2=""
    if ps!=[]:
        for p in ps:
            text=p.get_text()
            if text is not None and text not in content2:
                if text.count('。')>=1 and len(text)>minlen:
                    content2=content2+text
    ## compare
    if len(content)<=len(content2):
        content=content2
    ## filtering
    content=content.split("\n")
    longtext=""
    for c in content:
        if len(c)>=10 and ("。" in c or "！" in c or "？" in c):
            longtext=longtext+"\n"+c
    return longtext.replace("百度首页登录个人中心帐号设置意见反馈退出",'')
```


```python

### News.py
#!/usr/bin/python3
from bs4 import BeautifulSoup
import sqlite3
from funs import writeLog
from funs import getPage
from funs import getDate
from funs import getLongText
import time
import sys
## get page
def getSubPage(a,longtxt=1):
    ## string handler
    treatStr=lambda s: s.replace('\'','\'\'')
    ## nega words
    negaurlwords=('letv','youku','tudou','iqiyi','jfinfo','gmw.cn')
    negatitlewords=("(图)","（图）","车展","重磅","健康问答","养生","(组图)","（组图）","专治")
    ## url
    url=a.get("href")
    title=a.get_text().strip("\r\n ")
    if title!=None and url!=None:
        if url.find('http://')>=0 and len(title)>4:
            ## exist?
            sql="SELECT count(*) FROM news WHERE url=\'"+treatStr(url)+"\' LIMIT 1"
            cur.execute(sql)
            exist=cur.fetchall()[0][0]
            if exist==0:
                skip=0
                for w in negaurlwords:
                    if url.find(w)>=0:
                        skip=1
                        break
                if skip==0:
                    for w in negatitlewords:
                        if title.find(w)>=0:
                            skip=1
                            break
                if skip==0:
                    content=getLongText(url)
                    if content!=None and len(content)>=80:
                        return (getDate(),title,url,content)
    return None
## init database
conn=sqlite3.connect('News.db')
cur=conn.cursor()
createTable=("CREATE TABLE IF NOT EXISTS news ("
             "id INTEGER PRIMARY KEY AUTOINCREMENT, "
             "date DATE, "
             "title VARCHAR(500),"
             "url VARCHAR(800),"
             "content LONGTEXT);")
conn.execute(createTable)
## homepage
html=getPage("http://news.baidu.com")
bs=BeautifulSoup(html,"html.parser")
Divs=bs.find_all('div')
for d in Divs:
    if d.get("id")=="pane-news":
        links=d.find_all("a")
        for a in links:
            print(a)
            content=getSubPage(a,longtxt=2)
            if content!=None:
                ## write into database
                column=('date','title','url','content')
                sql="INSERT INTO news ("+','.join(column)+") VALUES("+','.join('?'*len(column))+");"
                cur.execute(sql,content)
                conn.commit()
## sub page
Times=int(sys.argv[1])
HomePages=(("http://finance.baidu.com",2),
    ("http://guonei.news.baidu.com",3),
    ("http://guoji.news.baidu.com",3),
    ("http://shehui.news.baidu.com",5),
    ("http://mil.news.baidu.com",6),
    ("http://tech.baidu.com",7),
    ("http://internet.baidu.com",7))
for page in HomePages:
    url=page[0]
    times=page[1]
    if Times%times==0:
        html=getPage(url)
        bs=BeautifulSoup(html,"html.parser")
        links=bs.find_all("a")
        for a in links:
            content=getSubPage(a)
            if content!=None:
                ## write into database
                column=('date','title','url','content')
                sql="INSERT INTO news ("+','.join(column)+") VALUES("+','.join('?'*len(column))+");"
                cur.execute(sql,content)
                conn.commit()

cur.close()
conn.close()

```

# 使用selenium爬取动态页面

现在，很多网页通过使用JavaScript的Ajax技术实现了网页内容的动态改变。

比如，一个很简单的例子，如果访问我们之前建立的web.py中的网址： http://127.0.0.1:5000/dynamic 上面会有一个按钮，每按一次，该页面就会向服务器再发一个请求，服务器收到请求后会随机产生一个数字，并返回。所以每一次点击按钮，网页内容都会动态改变。

在碰到动态页面时，一种方法是精通JavaScript，并使用浏览器跟踪浏览器行为，分析JavaScript脚本，进而使用以上的方法模拟浏览器请求。但是这种方法非常复杂，很多时候JavaScript可能会复杂到一定程度，使得分析异常困难。

而另外一种方法，即使用selenium直接调用浏览器，浏览器自动执行JavaScript，然后调用浏览器的HTML即可。这种方法非常方便，但是速度异常之慢。

为了实现这一方法，我们首先要安装selenium：pip install selenium

除此之外，还要安装浏览器的支持。几种常见的浏览器支持插件的下载地址：

* firefox：https://github.com/mozilla/geckodriver/releases （对于Windows，下载后放到C:\Windows下；对于Linux/Mac，放到/usr/local/bin下。）
* chrome： https://sites.google.com/a/chromium.org/chromedriver/downloads
* safari： https://webkit.org/blog/6900/webdriver-support-in-safari-10/
* edge：   https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/

安装完成之后，使用：
```python
from selenium import webdriver
driver=webdriver.Firefox()
## 做一些事情
driver.close()
```

就可以打开网络驱动器，此时我们可以看到一个Firefox窗口被打开。同样注意的是，用完之后记得关掉。

比如，淘宝的网页上，价格数据是动态加载的，我们可以使用如下代码找到价格：

In [1]:
from selenium import webdriver
import time
import re
import json
from bs4 import BeautifulSoup
url="https://detail.tmall.com/item.htm?id=524649065000&sku_properties=29112:97926"
driver=webdriver.Firefox()
driver.get(url)
html=driver.page_source
driver.close()
## 产品名
bs=BeautifulSoup(html,"html.parser")
meta=bs.find_all("meta")
for m in meta:
    if m.get("name")=="keywords":
        title=m.get("content")
## 价格
pattern=re.compile("TShop.Setup\(.+?\);")
res=pattern.finditer(html.replace('\r', '').replace('\n', ''))
for r in res:
    json_text=r.group()
    json_text=json_text[json_text.find('{'):-2]

info=json.loads(json_text)
print(title,info['detail']['defaultItemPrice'])

ModuleNotFoundError: No module named 'selenium'

selenium另外一个更常用的功能是自动填充、点击。为了实现这一目的，首先需要能够找到相应的按钮、输入框等，有如下方法可以使用：

* find_element_by_name
* find_element_by_id
* find_element_by_xpath
* find_element_by_link_text
* find_element_by_partial_link_text
* find_element_by_tag_name
* find_element_by_class_name
* find_element_by_css_selector

理解以上函数需要一些HTML、CSS、JavaScript的背景知识，我们在此不再详细讨论。在此，展示一个简单的例子：

In [2]:
from selenium import webdriver
import time
import re
import json
from bs4 import BeautifulSoup
url="http://weixin.sogou.com"
driver=webdriver.Firefox()
driver.get(url)
driver.find_element_by_id('query').send_keys("Python爬虫") #模仿填写搜索内容
driver.find_element_by_class_name("swz").click() #模仿点击搜索按钮
html=driver.page_source
driver.close()
bs=BeautifulSoup(html,"html.parser")
iterms=bs.find_all(name='h3')
for it in iterms:
    print(it.a.text,it.a['href'])

python爬虫:爬取拉勾网职位并分析 /link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6cwJThYulHEtVjXrGTiVgS8gCVKMYhvEenxvx7aByKuCvUlyE6x3M81qXa8Fplpd90jHmQgjRIXY7mIX539ZpwhFQM2OZCZvNMiSUIjcd_YfkSc8Cnn6iKsHvrmifn9NTHSnyL8YMIPQtOlBK4vjUwkY3UZ9ReBtA-0ZgXwy561hoQvhlM8xprtVo9-RYUDiAAEw80LtPIqexaAhVnIcPCEJB2Mn2lCVupnXWzMIwAA5Ae0f8bRARvQ..&type=2&query=Python%E7%88%AC%E8%99%AB
Python爬虫&可视化-舌尖上的“小龙虾” /link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6cwJThYulHEtVjXrGTiVgS8gCVKMYhvEenxvx7aByKuCvUlyE6x3M81qXa8Fplpd90jHmQgjRIXY7mIX539ZpwhFQM2OZCZvNMiSUIjcd_YfkSc8Cnn6iKj-kLLyU3EIjchL23SuZDu6z4ZhRfZuF31setyDrI2QIYQ20oM16JkG0BmnKwnscj2hzOcebM5tbS4zB0kyvqjcO0CBWeSh3bc-lR_CaTGWpuTc3GUoBBZHm0CtGSSkZQQ..&type=2&query=Python%E7%88%AC%E8%99%AB
Python爬虫 | 一条高效的学习路径 /link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6cwJThYulHEtVjXrGTiVgS8gCVKMYhvEenxvx7aByKuCvUlyE6x3M81qXa8Fplpd9gAjh8MJgnq3Mzykzf9tgIMS_sxdeItQSsYHwoxq1Fmdm_FsIoebRQwplAX_zKJc2nHGoOMhYq-yitSxruXv-Kx_Q1owGRniEr6VfCWQcc5sGQMi0p-AkjCrVPIETMbc3wOzOVLVeEN85jMbHJX5-7HXGLQVmIiovsw3Sjlqo_Zfd

# 后记：关于爬虫的一些补充

关于爬虫，这里有一些需要补充的话。

首先是关于爬虫的道德问题。实际上爬虫一直处在一个灰色地带：违法或者不违法，道德或者不道德。在此一些原则希望与大家分享：

* 按照我国法律的相关规定，如果网页内容、API接口使用了加密，破解加密是违法行为。
* 如果将爬取的数据商用，法律风险非常大
* 如果网站提供了接口（比如豆瓣API），不使用接口而直接爬取网页也是不道德的。
* 尽量爬取的速率不要太高，不要给他人的服务器造成太大负担。

当然，还有一些技术出于时间所限、个人能力所限，我是没有讲到的，这其中很多都是与反爬虫有关。比如：

* 如何破解验证码
* 如何破解短信验证码
* 消息队列
* 并行爬虫

等等。如果有需要，可以自行学习。

最后，爬虫和反爬一直是相互伴生的两个技术，我们这里提供一些常用的反爬思路：

* 如果request到的内容与自己使用浏览器看到的内容不符：
    - 首先查看请求头部，最极端情况下，完全复制浏览器的请求头部，看看能不能得到相同的反应。
    - 是否需要cookies登录？
    - 是否有重定向？
    - 仔细查看request得到的html，里面可能有玄机
    - 实在不行，selenium
* 如果request中的内容没有自己想要的信息：
    - 是否是动态加载的？
        * 对比request得到的HTML和浏览器的HTML是否不同，如果是不同的，可能时动态加载的
        * 仔细分析JavaScript，或者浏览器发送的每一次请求
        * 尝试直接发送动态请求
    - 内容是否是加密的？
        * 仔细分析加密方法
* 爬取几个页面后被禁止访问：
    - 可能爬取的频率太频繁
    - 间隔时间小一点
    - 使用代理池（比如该项目： https://github.com/SpiderClub/haipproxy ）
    - 使用ADSL
* 需要登录：
    - 不频繁的登录通常使用cookies，模拟一次登录之后获得cookies即可
    - 频繁的登录或者验证码，可能需要结合图像识别之类的方法